In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

In [12]:
data = pd.read_excel(r'C:\Users\basil\OneDrive\Documents\lab_4_data.xlsx')
testData = pd.read_excel(r'C:\Users\basil\OneDrive\Documents\lab_4_test_data.xlsx')
print(data)
d = data.pop('d')
dTest = testData.pop('d')

kval = 10
itertot = 40
sigma = 1.2
itergd = 300

        x1      x2      x3       d
0   0.9532  0.6949  0.4451  0.8426
1   0.7954  0.8346  0.0449  0.6676
2   0.1427  0.0480  0.6267  0.3780
3   0.1516  0.9824  0.0827  0.4627
4   0.4868  0.6223  0.7462  0.8116
..     ...     ...     ...     ...
95  0.6282  0.1404  0.8474  0.6733
96  0.5861  0.6693  0.3818  0.7433
97  0.6057  0.9901  0.5141  0.8466
98  0.5915  0.5588  0.3055  0.6787
99  0.8359  0.4145  0.5016  0.7597

[100 rows x 4 columns]


In [27]:
def transforminput(param, center):
	newinput = np.zeros((len(param), len(center))).astype('float32')
	for i in range(len(param)):
		for j in range(len(center)):
			newinput[i,j] = np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
	return newinput

def generatemodel(numparam):
	model = Sequential()
	model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
#	model.add(Dense(10, activation='sigmoid'))
#	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


tf_dataset = tf.data.Dataset.from_tensor_slices((data.values, d.values))
tf_test_dataset = tf.data.Dataset.from_tensor_slices((testData.values, dTest.values))
trainparam,trainlabel = [],[]
testparam,testlabel = [],[]
for feature,value in tf_dataset.as_numpy_iterator() :
    trainparam.append(feature)
    trainlabel.append(value)
    
for feature,value in tf_test_dataset.as_numpy_iterator() :
    testparam.append(feature)
    testlabel.append(value)

trainparam

[array([0.9532, 0.6949, 0.4451]),
 array([0.7954, 0.8346, 0.0449]),
 array([0.1427, 0.048 , 0.6267]),
 array([0.1516, 0.9824, 0.0827]),
 array([0.4868, 0.6223, 0.7462]),
 array([0.3408, 0.5115, 0.0783]),
 array([0.8146, 0.6378, 0.5837]),
 array([0.282 , 0.5409, 0.7256]),
 array([0.5716, 0.2958, 0.5477]),
 array([0.9323, 0.0229, 0.4797]),
 array([0.2907, 0.7245, 0.5165]),
 array([0.0068, 0.0545, 0.0861]),
 array([0.2636, 0.9885, 0.2175]),
 array([0.035 , 0.3653, 0.7801]),
 array([0.967 , 0.3031, 0.7127]),
 array([0.    , 0.7763, 0.8735]),
 array([0.4395, 0.0501, 0.9761]),
 array([0.9359, 0.0366, 0.9514]),
 array([0.0173, 0.9548, 0.4289]),
 array([0.6112, 0.907 , 0.6286]),
 array([0.201 , 0.9573, 0.6791]),
 array([0.8914, 0.9144, 0.2641]),
 array([0.0061, 0.0802, 0.8621]),
 array([0.2212, 0.4664, 0.3821]),
 array([0.2401, 0.6964, 0.0751]),
 array([0.7881, 0.9833, 0.3038]),
 array([0.2435, 0.0794, 0.5551]),
 array([0.2752, 0.8414, 0.2797]),
 array([0.7616, 0.4698, 0.5337]),
 array([0.3395

In [29]:
trainparam = np.asarray(trainparam)
testparam = np.asarray(testparam)

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
	std[i] = np.std(trainparam[:,i])
	rata[i] = np.mean(trainparam[:,i])
	trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
	testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

In [40]:
#init kmean
kmean = np.zeros((kval, len(trainparam[0])))

for i in range(kval):
	for j in range(len(kmean[0])):
		kmean[i,j] = random.uniform(min(trainparam[:,j]),max(trainparam[:,j]))

#looping of real algorithm
distmin = np.zeros((len(trainparam)))
for i in range(itertot):
	print ('iteration no', i)
	for j in range(len(distmin)):
		#determine euclid distance
		distall = np.sum((trainparam[j] - kmean)**2.0, axis=1)**0.5
		distmin[j] = np.argmin(distall)

	#search new k mean
	for j in range(kval):
		clust = []
		for k in range(len(distmin)):
			if distmin[k] == j:
				clust.append(trainparam[k])
		if len(clust) > 0:
			kmean[j] = np.mean(np.asarray(clust), axis=0)

#tranform our input
newinput = transforminput(trainparam, kmean)

print (trainlabel)

iteration no 0
iteration no 1
iteration no 2
iteration no 3
iteration no 4
iteration no 5
iteration no 6
iteration no 7
iteration no 8
iteration no 9
iteration no 10
iteration no 11
iteration no 12
iteration no 13
iteration no 14
iteration no 15
iteration no 16
iteration no 17
iteration no 18
iteration no 19
iteration no 20
iteration no 21
iteration no 22
iteration no 23
iteration no 24
iteration no 25
iteration no 26
iteration no 27
iteration no 28
iteration no 29
iteration no 30
iteration no 31
iteration no 32
iteration no 33
iteration no 34
iteration no 35
iteration no 36
iteration no 37
iteration no 38
iteration no 39
[0.8426 0.6676 0.378  0.4627 0.8116 0.4559 0.8628 0.6939 0.6619 0.5731
 0.6911 0.0851 0.5847 0.5117 0.7836 0.6388 0.5712 0.6826 0.5527 0.8803
 0.7283 0.7966 0.3711 0.526  0.4637 0.8049 0.4223 0.6079 0.7809 0.1836
 0.8641 0.4857 0.6667 0.7829 0.8016 0.5449 0.8991 0.9047 0.3561 0.331
 0.6245 0.6066 0.7172 0.3385 0.159  0.8254 0.9782 0.325  0.5404 0.936
 0.6949 0.5625 0.

In [41]:
trainlabel = np.asarray(trainlabel)
testlabel = np.asarray(testlabel)
mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)	

Epoch 1/300
5/5 [==============================] - 0s 1ms/step - loss: 0.6870 - accuracy: 0.0000e+00
Epoch 2/300
5/5 [==============================] - 0s 992us/step - loss: 0.6837 - accuracy: 0.0000e+00
Epoch 3/300
5/5 [==============================] - 0s 868us/step - loss: 0.6802 - accuracy: 0.0000e+00
Epoch 4/300
5/5 [==============================] - 0s 993us/step - loss: 0.6775 - accuracy: 0.0000e+00
Epoch 5/300
5/5 [==============================] - 0s 868us/step - loss: 0.6745 - accuracy: 0.0000e+00
Epoch 6/300
5/5 [==============================] - 0s 992us/step - loss: 0.6726 - accuracy: 0.0000e+00
Epoch 7/300
5/5 [==============================] - 0s 868us/step - loss: 0.6705 - accuracy: 0.0000e+00
Epoch 8/300
5/5 [==============================] - 0s 868us/step - loss: 0.6685 - accuracy: 0.0000e+00
Epoch 9/300
5/5 [==============================] - 0s 868us/step - loss: 0.6669 - accuracy: 0.0000e+00
Epoch 10/300
5/5 [==============================] - 0s 868us/step - loss: 0

In [42]:
newinputtest = transforminput(testparamnorm, kmean)

lifeprob = mod.predict(newinputtest)

2/2 [==============================] - 0s 1ms/step


In [43]:
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
	if lifeprob[i] > 0.5:
		binpred[i] = 1.

score = 0
for i in range(len(testlabel)):
	if binpred[i] == testlabel[i]:
		score += 1
accbin = float(score) / float(len(testlabel))

#determine brier score
brierscore = 0
for i in range(len(testlabel)):
	brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))

for i in range(len(testlabel)):
	print (lifeprob[i], testlabel[i])
print (accbin, brierscore[0])

[0.5908201] 0.8521
[0.4893542] 0.5772
[0.48598343] 0.6211
[0.5386371] 0.7693
[0.5398314] 0.7878
[0.46727183] 0.5777
[0.4572259] 0.4261
[0.46346414] 0.4625
[0.48171598] 0.6015
[0.47538644] 0.5739
[0.46066654] 0.4007
[0.5824901] 0.9141
[0.47887275] 0.4707
[0.6422529] 0.822
[0.5034631] 0.698
[0.4773007] 0.4851
[0.4813338] 0.4802
[0.48907372] 0.7215
[0.46717572] 0.5913
[0.52290034] 0.6805
[0.4856169] 0.6033
[0.61986583] 0.8226
[0.45962292] 0.524
[0.50509953] 0.7143
[0.46269932] 0.4547
[0.47532186] 0.4602
[0.48428762] 0.5477
[0.47239465] 0.4738
[0.5111145] 0.7397
[0.46984676] 0.4526
[0.5226892] 0.7711
[0.47381815] 0.5571
[0.45742923] 0.3759
[0.47218943] 0.4018
[0.5139144] 0.6563
[0.66989815] 0.9128
[0.4585919] 0.3055
[0.6597837] 0.9298
[0.4828433] 0.5377
[0.56653446] 0.7829
[0.47273052] 0.3944
[0.49581382] 0.5988
[0.62180877] 0.9728
[0.5031308] 0.6832
[0.46676144] 0.4622
[0.58524716] 0.8191
[0.4701618] 0.4765
[0.46504387] 0.4953
[0.4780539] 0.6376
[0.456222] 0.4276
0.0 0.024879498
